In [ ]:
#로또번호 예측 트랜스포머

import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
import numpy as np

# CSV 파일 불러오기
data = pd.read_csv('lotto.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-6]  # 마지막 6개 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터


# 데이터셋 클래스 정의
class LottoDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        return self.inputs[index], self.labels[index]

# 트랜스포머 모델 정의
class LottoTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(LottoTransformer, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(input_size, output_size)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 하이퍼파라미터 설정
input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]''
num_layers = 4
num_heads = 4
num_epochs = 1000
learning_rate = 0.001
batch_size = 32

# 데이터 로더 초기화
dataset = LottoDataset(train_inputs, train_labels)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 모델 초기화
model = LottoTransformer(input_size, hidden_size, output_size, num_layers, num_heads)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 모델 훈련
for epoch in range(num_epochs):
    model.train()
    for batch_inputs, batch_labels in data_loader:
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 테스트 데이터 준비
test_inputs = torch.tensor(scaled_data[-3:-1]).unsqueeze(0).float()

# 모델 예측
model.eval()
with torch.no_grad():
    predicted = model(test_inputs)

# 스케일링 역변환
predicted = scaler.inverse_transform(predicted.squeeze(0).numpy())

# 예측 결과 출력
lotto_numbers_prediction = predicted[-1].astype(int)[:-5]  # 마지막 5개 원소 제외
print(f'2023년 6월 로또 번호 예측: {lotto_numbers_prediction}')

Epoch [100/1000], Loss: 0.0378
Epoch [200/1000], Loss: 0.0332
Epoch [300/1000], Loss: 0.0306
Epoch [400/1000], Loss: 0.0289
Epoch [500/1000], Loss: 0.0279
Epoch [600/1000], Loss: 0.0277
Epoch [700/1000], Loss: 0.0273
Epoch [800/1000], Loss: 0.0272
Epoch [900/1000], Loss: 0.0272
Epoch [1000/1000], Loss: 0.0270
2023년 6월 로또 번호 예측: [ 6 12 19 26 32 39 22]


In [ ]:
#코스피 주가지수 예측 트랜스포머 (기본)

# CSV 파일 불러오기
data = pd.read_csv('data100.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(input_size, output_size)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 하이퍼파라미터 설정
input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6 #로또와 달리 데이터셋이 커 layer 4에서 6증가
num_heads = 4
num_epochs = 1000 #찾는 횟수
learning_rate = 0.001

# 모델 초기화
model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 모델 훈련
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(train_inputs)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 테스트 데이터 준비
test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()

# 모델 예측
model.eval()
with torch.no_grad():
    predicted = model(test_inputs)

# 스케일링 역변환
predicted = scaler.inverse_transform(predicted.squeeze().unsqueeze(0).numpy())

# 예측 결과 출력
kospis_index_prediction = predicted[0][-1]

print(f'2023년 4월의 코스피지수 예측: {kospis_index_prediction:.2f}')


Epoch [100/1000], Loss: 0.0216
Epoch [200/1000], Loss: 0.0108
Epoch [300/1000], Loss: 0.0072
Epoch [400/1000], Loss: 0.0056
Epoch [500/1000], Loss: 0.0047
Epoch [600/1000], Loss: 0.0041
Epoch [700/1000], Loss: 0.0036
Epoch [800/1000], Loss: 0.0032
Epoch [900/1000], Loss: 0.0029
Epoch [1000/1000], Loss: 0.0026
2023년 4월의 코스피지수 예측: 2266.60


In [ ]:
#코스피 주가지수 예측 트랜스포머 (10회 실행 후 평균 및 표준편차)

# CSV 파일 불러오기
data = pd.read_csv('data100.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(input_size, output_size)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 하이퍼파라미터 설정
input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6
num_heads = 4
num_runs = 10  # 실행 횟수
num_epochs = 1000
learning_rate = 0.001

predictions = []  # 예측 결과를 저장할 리스트

for _ in range(num_runs):
    # 모델 초기화
    model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)

    # 손실 함수와 옵티마이저 정의
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # 모델 훈련
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs)
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # 테스트 데이터 준비
    test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()

    # 모델 예측
    model.eval()
    with torch.no_grad():
        predicted = model(test_inputs)

    # 스케일링 역변환
    predicted = scaler.inverse_transform(predicted.squeeze().unsqueeze(0).numpy())

    # 예측 결과 저장
    kospis_index_prediction = predicted[0][-1]
    predictions.append(kospis_index_prediction)

# 예측 결과 평균 및 표준 편차 계산
predictions = np.array(predictions)
mean_prediction = np.mean(predictions)
std_prediction = np.std(predictions)

print(f'Mean of Predicted KOSPI Index: {mean_prediction:.2f}')
print(f'Standard Deviation of Predicted KOSPI Index: {std_prediction:.2f}')


Epoch [100/1000], Loss: 0.0200
Epoch [200/1000], Loss: 0.0106
Epoch [300/1000], Loss: 0.0073
Epoch [400/1000], Loss: 0.0057
Epoch [500/1000], Loss: 0.0047
Epoch [600/1000], Loss: 0.0040
Epoch [700/1000], Loss: 0.0036
Epoch [800/1000], Loss: 0.0031
Epoch [900/1000], Loss: 0.0028
Epoch [1000/1000], Loss: 0.0027
Epoch [100/1000], Loss: 0.0215
Epoch [200/1000], Loss: 0.0102
Epoch [300/1000], Loss: 0.0073
Epoch [400/1000], Loss: 0.0059
Epoch [500/1000], Loss: 0.0049
Epoch [600/1000], Loss: 0.0041
Epoch [700/1000], Loss: 0.0036
Epoch [800/1000], Loss: 0.0032
Epoch [900/1000], Loss: 0.0029
Epoch [1000/1000], Loss: 0.0027
Epoch [100/1000], Loss: 0.0182
Epoch [200/1000], Loss: 0.0103
Epoch [300/1000], Loss: 0.0072
Epoch [400/1000], Loss: 0.0056
Epoch [500/1000], Loss: 0.0046
Epoch [600/1000], Loss: 0.0039
Epoch [700/1000], Loss: 0.0035
Epoch [800/1000], Loss: 0.0032
Epoch [900/1000], Loss: 0.0028
Epoch [1000/1000], Loss: 0.0026
Epoch [100/1000], Loss: 0.0239
Epoch [200/1000], Loss: 0.0113
Epoch

In [4]:
#가중치가 반영된 코스피 주가지수 예측 트랜스포머

# CSV 파일 불러오기
data = pd.read_csv('data100.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x, weights):
        encoded = self.encoder(x)
        weighted_encoded = encoded * weights.unsqueeze(0).unsqueeze(1).unsqueeze(1).expand(-1, encoded.size(1), -1, -1).transpose(0, 1)
        decoded = self.decoder(weighted_encoded)
        return decoded

# 하이퍼파라미터 설정
input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6
num_heads = 4
num_epochs = 1000
learning_rate = 0.001

# 모델 초기화
model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 가중치 설정
weights = torch.randn(hidden_size)

# 모델 훈련
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(train_inputs, weights)  # 가중치 전달
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 테스트 데이터 준비
test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()

# 모델 예측
model.eval()
with torch.no_grad():
    predicted = model(test_inputs, weights)  # 가중치 전달

# 스케일링 역변환
predicted = scaler.inverse_transform(predicted.squeeze(0).transpose(1, 3).reshape(-1, predicted.shape[-1])).reshape(-1, predicted.shape[-1])

# 예측 결과 출력
kospis_index_prediction = predicted[-1, -1]
print(f'Predicted KOSPI Index: {kospis_index_prediction:.2f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0229
Epoch [200/1000], Loss: 0.0120
Epoch [300/1000], Loss: 0.0082
Epoch [400/1000], Loss: 0.0063
Epoch [500/1000], Loss: 0.0052
Epoch [600/1000], Loss: 0.0047
Epoch [700/1000], Loss: 0.0041
Epoch [800/1000], Loss: 0.0038
Epoch [900/1000], Loss: 0.0034
Epoch [1000/1000], Loss: 0.0031
Predicted KOSPI Index: 2259.52


In [5]:
#코스피 주가지수의 증감 퍼센트를 확인하는 트랜스포머

# CSV 파일 불러오기
data = pd.read_csv('percent.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x, weights):
        encoded = self.encoder(x)
        weighted_encoded = encoded * weights.unsqueeze(0).unsqueeze(1).unsqueeze(1).expand(-1, encoded.size(1), -1, -1).transpose(0, 1)
        decoded = self.decoder(weighted_encoded)
        return decoded

input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6
num_heads = 4
num_epochs = 1000
learning_rate = 0.001
num_runs = 10

predictions = []

for _ in range(num_runs):

    model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)


    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


    weights = torch.randn(hidden_size)

    # 모델 훈련
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs, weights)  # 가중치 전달
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # 테스트 데이터 준비
    test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()

    # 모델 예측
    model.eval()
    with torch.no_grad():
        predicted = model(test_inputs, weights)  # 가중치 전달

    # 스케일링 역변환
    predicted = scaler.inverse_transform(predicted.squeeze(0).transpose(1, 3).reshape(-1, predicted.shape[-1])).reshape(-1, predicted.shape[-1])

    # 예측 결과 저장
    predictions.append(predicted[-1, -1])

# 예측 결과의 평균과 표준 편차 계산
predictions = np.array(predictions)
mean_prediction = predictions.mean()
std_prediction = predictions.std()

print(f'Mean of Predicted KOSPI Index: {mean_prediction:.2f}')
print(f'Standard Deviation of Predicted KOSPI Index: {std_prediction:.2f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0216
Epoch [200/1000], Loss: 0.0200
Epoch [300/1000], Loss: 0.0188
Epoch [400/1000], Loss: 0.0178
Epoch [500/1000], Loss: 0.0166
Epoch [600/1000], Loss: 0.0150
Epoch [700/1000], Loss: 0.0136
Epoch [800/1000], Loss: 0.0120
Epoch [900/1000], Loss: 0.0106
Epoch [1000/1000], Loss: 0.0094


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0217
Epoch [200/1000], Loss: 0.0204
Epoch [300/1000], Loss: 0.0193
Epoch [400/1000], Loss: 0.0190
Epoch [500/1000], Loss: 0.0179
Epoch [600/1000], Loss: 0.0170
Epoch [700/1000], Loss: 0.0156
Epoch [800/1000], Loss: 0.0145
Epoch [900/1000], Loss: 0.0130
Epoch [1000/1000], Loss: 0.0120


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Epoch [100/1000], Loss: 0.0209
Epoch [200/1000], Loss: 0.0193
Epoch [300/1000], Loss: 0.0186
Epoch [400/1000], Loss: 0.0173
Epoch [500/1000], Loss: 0.0158
Epoch [600/1000], Loss: 0.0141
Epoch [700/1000], Loss: 0.0127
Epoch [800/1000], Loss: 0.0113
Epoch [900/1000], Loss: 0.0101
Epoch [1000/1000], Loss: 0.0091


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0210
Epoch [200/1000], Loss: 0.0193
Epoch [300/1000], Loss: 0.0183
Epoch [400/1000], Loss: 0.0177
Epoch [500/1000], Loss: 0.0160
Epoch [600/1000], Loss: 0.0148
Epoch [700/1000], Loss: 0.0132
Epoch [800/1000], Loss: 0.0117
Epoch [900/1000], Loss: 0.0105
Epoch [1000/1000], Loss: 0.0093


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0215
Epoch [200/1000], Loss: 0.0198
Epoch [300/1000], Loss: 0.0192
Epoch [400/1000], Loss: 0.0186
Epoch [500/1000], Loss: 0.0175
Epoch [600/1000], Loss: 0.0168
Epoch [700/1000], Loss: 0.0155
Epoch [800/1000], Loss: 0.0141
Epoch [900/1000], Loss: 0.0127
Epoch [1000/1000], Loss: 0.0114
Mean of Predicted KOSPI Index: -3.36
Standard Deviation of Predicted KOSPI Index: 3.06


In [ ]:
#코스피 주가지수의 증감 퍼센트 및 가중치와 10회 실행 후 평균과 표준편차를 구하는 트랜스포머

# CSV 파일 불러오기
data = pd.read_csv('percent.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x, weights):
        encoded = self.encoder(x)
        weighted_encoded = encoded * weights.unsqueeze(0).unsqueeze(1).unsqueeze(1).expand(-1, encoded.size(1), -1, -1).transpose(0, 1)
        decoded = self.decoder(weighted_encoded)
        return decoded

input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6
num_heads = 4
num_epochs = 1000
learning_rate = 0.001
num_runs = 10

predictions = []

for _ in range(num_runs):

    model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)


    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


    weights = torch.randn(hidden_size)

    # 모델 훈련
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs, weights)  # 가중치 전달
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # 테스트 데이터 준비
    test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()

    # 모델 예측
    model.eval()
    with torch.no_grad():
        predicted = model(test_inputs, weights)  # 가중치 전달

    # 스케일링 역변환
    predicted = scaler.inverse_transform(predicted.squeeze(0).transpose(1, 3).reshape(-1, predicted.shape[-1])).reshape(-1, predicted.shape[-1])

    # 예측 결과 저장
    predictions.append(predicted[-1, -1])

# 예측 결과의 평균과 표준 편차 계산
predictions = np.array(predictions)
mean_prediction = predictions.mean()
std_prediction = predictions.std()

print(f'Mean of Predicted KOSPI Index: {mean_prediction:.2f}')
print(f'Standard Deviation of Predicted KOSPI Index: {std_prediction:.2f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0224
Epoch [200/1000], Loss: 0.0200
Epoch [300/1000], Loss: 0.0192
Epoch [400/1000], Loss: 0.0181
Epoch [500/1000], Loss: 0.0168
Epoch [600/1000], Loss: 0.0155
Epoch [700/1000], Loss: 0.0140
Epoch [800/1000], Loss: 0.0126
Epoch [900/1000], Loss: 0.0112
Epoch [1000/1000], Loss: 0.0101


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0216
Epoch [200/1000], Loss: 0.0198
Epoch [300/1000], Loss: 0.0186
Epoch [400/1000], Loss: 0.0178
Epoch [500/1000], Loss: 0.0171
Epoch [600/1000], Loss: 0.0162
Epoch [700/1000], Loss: 0.0152
Epoch [800/1000], Loss: 0.0142
Epoch [900/1000], Loss: 0.0126
Epoch [1000/1000], Loss: 0.0112


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Epoch [100/1000], Loss: 0.0211
Epoch [200/1000], Loss: 0.0195
Epoch [300/1000], Loss: 0.0188
Epoch [400/1000], Loss: 0.0174
Epoch [500/1000], Loss: 0.0159
Epoch [600/1000], Loss: 0.0142
Epoch [700/1000], Loss: 0.0123
Epoch [800/1000], Loss: 0.0107
Epoch [900/1000], Loss: 0.0093
Epoch [1000/1000], Loss: 0.0084


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0204
Epoch [200/1000], Loss: 0.0190
Epoch [300/1000], Loss: 0.0178
Epoch [400/1000], Loss: 0.0162
Epoch [500/1000], Loss: 0.0142
Epoch [600/1000], Loss: 0.0123
Epoch [700/1000], Loss: 0.0109
Epoch [800/1000], Loss: 0.0095
Epoch [900/1000], Loss: 0.0084
Epoch [1000/1000], Loss: 0.0076


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 0.0219
Epoch [200/1000], Loss: 0.0202
Epoch [300/1000], Loss: 0.0188
Epoch [400/1000], Loss: 0.0173
Epoch [500/1000], Loss: 0.0163
Epoch [600/1000], Loss: 0.0146
Epoch [700/1000], Loss: 0.0132
Epoch [800/1000], Loss: 0.0120
Epoch [900/1000], Loss: 0.0110
Epoch [1000/1000], Loss: 0.0099
Mean of Predicted KOSPI Index: -2.57
Standard Deviation of Predicted KOSPI Index: 3.29


In [6]:
#위 코드에서 loss함수를 Huber Loss로 대체한 코드

# CSV 파일 불러오기
data = pd.read_csv('percent.csv')

# 데이터 전처리
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.iloc[:, 1:].values)  # 날짜 열 제외, 수치 데이터 스케일링

# 훈련 데이터 준비
train_data = scaled_data[:-1]  # 마지막 행 제외
train_inputs = torch.tensor(train_data[:-1]).unsqueeze(0).float()  # 입력 데이터
train_labels = torch.tensor(train_data[1:]).unsqueeze(0).float()  # 출력 데이터

# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_size, num_heads, hidden_size),
            num_layers
        )
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x, weights):
        encoded = self.encoder(x)
        weighted_encoded = encoded * weights.unsqueeze(0).unsqueeze(1).unsqueeze(1).expand(-1, encoded.size(1), -1, -1).transpose(0, 1)
        decoded = self.decoder(weighted_encoded)
        return decoded

input_size = train_inputs.shape[-1]
hidden_size = 64
output_size = train_labels.shape[-1]
num_layers = 6
num_heads = 4
num_epochs = 1000
learning_rate = 0.001
num_runs = 2

predictions = []

for _ in range(num_runs):

    model = TransformerModel(input_size, hidden_size, output_size, num_layers, num_heads)

    criterion = nn.SmoothL1Loss()  # Huber Loss로 대체

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    weights = torch.randn(hidden_size)

    # 모델 훈련
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_inputs, weights)  # 가중치 전달
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # 테스트 데이터 준비
    test_inputs = torch.tensor(scaled_data[-2]).unsqueeze(0).float()

    # 모델 예측
    model.eval()
    with torch.no_grad():
        predicted = model(test_inputs, weights)  # 가중치 전달

    # 스케일링 역변환
    predicted = scaler.inverse_transform(predicted.squeeze(0).transpose(1, 3).reshape(-1, predicted.shape[-1])).reshape(-1, predicted.shape[-1])

    # 예측 결과 저장
    predictions.append(predicted[-1, -1])

# 예측 결과의 평균과 표준 편차 계산
predictions = np.array(predictions)
mean_prediction = predictions.mean()
std_prediction = predictions.std()

print(f'Mean of Predicted KOSPI Index: {mean_prediction:.2f}')
print(f'Standard Deviation of Predicted KOSPI Index: {std_prediction:.2f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


Epoch [100/1000], Loss: 0.0106
Epoch [200/1000], Loss: 0.0097
Epoch [300/1000], Loss: 0.0093
Epoch [400/1000], Loss: 0.0088
Epoch [500/1000], Loss: 0.0084
Epoch [600/1000], Loss: 0.0077
Epoch [700/1000], Loss: 0.0069
Epoch [800/1000], Loss: 0.0060
Epoch [900/1000], Loss: 0.0053
Epoch [1000/1000], Loss: 0.0047


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([1, 217, 64])) that is different to the input size (torch.Size([217, 1, 217, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


Epoch [100/1000], Loss: 0.0105
Epoch [200/1000], Loss: 0.0096
Epoch [300/1000], Loss: 0.0090
Epoch [400/1000], Loss: 0.0082
Epoch [500/1000], Loss: 0.0074
Epoch [600/1000], Loss: 0.0065
Epoch [700/1000], Loss: 0.0058
Epoch [800/1000], Loss: 0.0051
Epoch [900/1000], Loss: 0.0045
Epoch [1000/1000], Loss: 0.0043
Mean of Predicted KOSPI Index: -0.54
Standard Deviation of Predicted KOSPI Index: 0.10
